# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [35]:
# imports
from openai import OpenAI
from IPython.display import Markdown, display

In [36]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'
MODEL_GEMMA = 'google/gemma-3-12b'

In [37]:
# set up environment

In [38]:
# here is the question; type over this to ask something new

question = """
Пожалуйста, объясните, что делает этот код и почему:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [39]:
# Get gemma-3-12b to answer, with streaming
gemma = OpenAI(base_url='http://localhost:1234/v1', api_key='google/gemma-3-12b')
response = gemma.chat.completions.create(
    model=MODEL_GEMMA,
    messages=[{"role": "user", "content": question}],
)

display(Markdown(response.choices[0].message.content))

Этот код использует генераторное выражение с `yield from` для извлечения авторов из списка книг, но только тех, у которых есть информация об авторе. Давайте разберем его по частям:

**1. `books`:**

Предполагается, что `books` - это список словарей (или другой итерируемый объект), где каждый словарь представляет книгу.  Каждый словарь книги содержит ключи и значения, описывающие эту книгу. Например:

```python
books = [
    {"title": "The Lord of the Rings", "author": "J.R.R. Tolkien"},
    {"title": "Pride and Prejudice", "author": "Jane Austen"},
    {"title": "1984"},  # Нет информации об авторе
    {"title": "To Kill a Mockingbird", "author": "Harper Lee"}
]
```

**2. `book.get("author")`:**

*   `book` представляет текущую книгу в списке `books`.
*   `book.get("author")` пытается получить значение ключа `"author"` из словаря `book`.  Использование `.get()` вместо прямого доступа через `book["author"]` безопасно, потому что если ключ `"author"` отсутствует в словаре `book`, `book.get("author")` вернет `None` (или указанное значение по умолчанию, если оно предоставлено как второй аргумент для `get()`), а не вызовет ошибку `KeyError`.

**3. `for book in books if book.get("author")`:**

Это генераторное выражение с условием.  Оно перебирает каждую книгу в списке `books`, но *только* если `book.get("author")` возвращает непустое значение (т.е., не `None`, не пустую строку, не 0 и т.д.). Это гарантирует, что мы обрабатываем только те книги, у которых есть информация об авторе.

**4. `{... for ... if ...}`:**

Это генераторное выражение.  Оно создает последовательность значений (в данном случае, имена авторов) на основе итерации по `books`.  В отличие от списочного включения (`[... for ...]`), генераторные выражения не создают сразу весь список в памяти. Вместо этого они генерируют значения "по требованию", что более эффективно для больших объемов данных.

**5. `yield from ...`:**

*   `yield from` - это ключевое слово, которое позволяет делегировать выполнение другому итератору (в данном случае, генераторному выражению).
*   В этом контексте, `yield from {book.get("author") for book in books if book.get("author")}` означает: "Для каждого значения, генерируемого генераторным выражением `{book.get("author") for book in books if book.get("author")} `, передай это значение как часть текущего генератора".

**В итоге:**

Этот код создает генератор, который итерируется по списку книг `books` и выдает имена авторов только для тех книг, у которых есть информация об авторе.  Он делает это эффективно, не создавая сразу весь список имен авторов в памяти.

**Пример работы:**

Если `books` определен как показано выше:

```python
authors = yield from {book.get("author") for book in books if book.get("author")}

for author in authors:
    print(author)
```

Вывод будет:

```
J.R.R. Tolkien
Jane Austen
Harper Lee
```

**Почему `yield from`?**

Использование `yield from` делает код более читаемым и эффективным, особенно когда вы работаете с вложенными генераторами или итераторами.  Без `yield from`, вам пришлось бы написать что-то вроде:

```python
def get_authors(books):
    for book in books:
        author = book.get("author")
        if author:
            yield author
```

`yield from` позволяет избежать написания дополнительного цикла и условного оператора, делая код более компактным и понятным.  Он также может быть немного эффективнее, поскольку он позволяет Python напрямую делегировать выполнение генератору, не создавая промежуточные объекты.

В заключение, этот код представляет собой элегантный способ извлечения информации об авторах из списка книг с использованием генераторного выражения и `yield from` для повышения эффективности и читаемости.


In [40]:
# Get Llama 3.2 to answer
llama = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
response = llama.chat.completions.create(
    model=MODEL_LLAMA,
    messages=[{"role": "user", "content": question}],
)

display(Markdown(response.choices[0].message.content))

Этот код использования функционального шаблона (Generator expression) в Python.

Давайте разберем его поэтапно:

```python
from typing import Generator

class Book:
    def __init__(self, title: str, author: str):
        self.title = title
        self.author = author

books = [
    Book("Красный хорź", "МихAIL ПУШ КИРХ"),
    Book("Кукушkin", "а. к.),"),
    Book("Первородний Соколов Олексій Микольович",'),
]

def get_authors() -> Generator[str, None, None]:
    for book in books:
        author = book.get("author")
        if author is not None:
            yield from {author}
```

Эта функция `get_authors()` возвращает генератор (Generator), который можно использовать итерационно для отображения авторов книг `books`.

Работает она следующим образом: 

1. Сначала цикл `for` итерирует по sáchам.
2. Для каждое из них получается作者, если значение `"author"` в списке book не равно none.
3. Чтобы получить список элементов из множества "автор", используем функцию 'yield from'.

Суммируя все это, то `yield from {book.get("author") for book in books if book.get("author")}` — генератор, который проверяет каждую книгу в списке `books`. Если в списке есть хотя бы одна запись с `'author'` и они не равны None, то в результате получается множество с авторами.